In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import hashlib
import plotly.express as px

In [ ]:

df = pd.read_csv(r"athlete_events.csv")


In [ ]:
print(df['NOC'].nunique())
print(df['NOC'])

In [ ]:
df['Sport']

 e. ta reda på statistik för åldern: medelvärde, median, min, max, standardavvikelse,

In [ ]:
print(df['Age'].mean())
print(df['Age'].median())
print(df['Age'].min())
print(df['Age'].max())
print(df['Age'].std())

In [ ]:
df = df.fillna({
    'Age': df['Age'].median(),
    'Height': df['Height'].median(),
    'Weight': df['Weight'].median(),
    
})
df = df.astype({'Age': 'float32', 'Height': 'float32', 'Weight': 'float32', 'Year': 'int16'})
df.info()

In [ ]:
df['Sex'].value_counts().plot(kind='bar', color=['blue', 'red'])

In [ ]:

gender_year_season = df.groupby(['Season', 'Year', 'Sex']).size().unstack(fill_value=0)


fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharex=True)

gender_year_season.loc['Summer'].plot(ax=axes[0], marker='o')
axes[0].set_title('Summer OS')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Nbr of athletes')
axes[0].legend()
axes[0].grid(True)

gender_year_season.loc['Winter'].plot(ax=axes[1], marker='o')
axes[1].set_title('Winter OS')
axes[1].set_xlabel('Year')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
df.groupby('NOC')['Medal'].count().sort_values(ascending=False).iloc[:10].plot.bar()

# ITALIA

In [ ]:
italydf = df[df['NOC'] == 'ITA']
italydf

In [ ]:

#Skapar en Name_HASH kolumn med den anonymiserade namn
italydf.insert(loc=2, column="Name_HASH", value = italydf["Name"].apply( lambda x:
                                             hashlib.sha256(x.encode()).hexdigest()
                                             ))

#Skapar en anonym version av df utan namn-kolumn
italydf_anon = italydf.drop(["Name"], axis=1)

#Vi ska göra samma sak för den globala df
df.insert(loc=2, column="Name_HASH", value = df["Name"].apply( lambda x:
                                             hashlib.sha256(x.encode()).hexdigest()
                                             ))
df_anon = df.drop(["Name"], axis=1)
df_anon.head()



## Simning

### Medaljer

In [ ]:
#Först extraherar vi data för simning
ita_simn = italydf_anon[italydf_anon["Sport"]=="Swimming"]
ita_simn.head()

#och börjar kolla på medaljer och att filtrera ut dubletter 
ita_simn_medaljer = ita_simn.dropna(subset=["Medal"]).drop_duplicates(subset=["Year","Medal","Event","ID"])

#Nu gruperar vi medaljer per år
medaljer_år = ita_simn_medaljer.groupby("Year")["Medal"].count().reset_index()


In [ ]:

#Nu kan vi skapa vår första plot med plotly
fig = px.line(
    medaljer_år,
    x= "Year",
    y= "Medal",
    title="Antal Medaljer för Italien per OS",
    markers = True,
    )

fig.update_layout(
    xaxis_title = "OS År",
    yaxis_title = "Antal Medaljer"
)

fig.show()

In [ ]:
#Vi ska också titta på vilka olika medaljer italjen vann
medaljer_typ= ita_simn_medaljer.groupby(["Year","Medal"]).size().reset_index(name="Count")
medaljer_typ.head()

#Färgen ska matcha medaljen
color_map = {
    "Gold": "#C6A907",
    "Silver": "#C0C0C0",
    "Bronze": "#CD7F32"
}

fig = px.bar(
    medaljer_typ,
    x = "Year",
    y = "Count",
    color = "Medal",
    color_discrete_map=color_map,
    title= "Medaljfördelning för Italien i Simning per OS-år",
    barmode="group"
)

fig.update_layout(
    xaxis_title = "OS År",
    yaxis_title = "Antal Medaljer"

)
fig.show()

### Ålder

In [ ]:
#Först skapar vi ålder-dataset
ita_simn_ålder = ita_simn[ita_simn["Age"]>0]

In [ ]:
fig = px.histogram(
    ita_simn_ålder,
    x = "Age",
    nbins=20,
    title="Ålderfördelning för italienska simmare"
)
fig.update_layout(
    xaxis_title="Simmarens Åldern",
    yaxis_title= "Antal Atleter"
)
fig.show()

In [ ]:
#Nu ska vi kolla åldersdistribution per OS-år
fig = px.box(
    ita_simn_ålder,
    x="Year",
    y="Age",
    title = "Åldrar av italienska simmare per OS-år"
)

fig.update_layout(
    xaxis_title ="OS-År",
    yaxis_title="Ålder"
)
fig.show()

### Medaljer vs. Ålder

In [ ]:
fig = px.density_heatmap(
    ita_simn_medaljer,
    x = "Year",
    y = "Age",
    title="Medaljvinnande Simmare: Ålder och År (Densitet)",
    nbinsx=len(ita_simn_medaljer['Year']),
    nbinsy=20,
    color_continuous_scale="plasma"
)
fig.update_layout(
    xaxis_title="OS År",
    yaxis_title="Ålder",
)
fig.show()

### Simning Vs Andra Länder - Hur gick det för Italien

In [ ]:
#Nu kollar vi hur italien gjorde genomfort med resten av världen
#Först skapar vi en df frö alla simning medaljer i alla världen
global_simn = df_anon[
    (df_anon["Sport"]=="Swimming") &
    (df_anon["Medal"].notna())
     ].drop_duplicates(subset=["Year", "Event", "Medal", "ID"])

#Sen grouperar vi medaljerna i länder
medaljer_land = (
    global_simn.groupby("NOC")["Medal"]
    .count()
    .reset_index(name="Antal Medaljer")
    )

#vi skapar en färg kolumn så att Italien har sin egen färg
medaljer_land["Färg"]= medaljer_land["NOC"].apply(lambda x: "blue" if x == "ITA" else "red")

#Och i slutet sorterar vi vår data, och ska kolla på de högsta 20 länder
medaljer_land = medaljer_land.sort_values("Antal Medaljer", ascending=False).head(20)

In [ ]:
#Äntligen får vi skapa vår plot
fig = px.bar(
    medaljer_land,
    x="NOC",
    y="Antal Medaljer",
    color="Färg",
    color_discrete_map={"blue":"blue", "red":"red"},
    title = "Top 20 Världens Simning Medaljer"
)

fig.update_layout(
    xaxis_title="Land (NOC)",
    yaxis_title="Antal Medaljer",
    showlegend = False,
    xaxis={'categoryorder':'array', 'categoryarray': medaljer_land["NOC"]}
)

fig.show()